In [1]:
from datasets import *
from models_baselines import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from torch.utils.data import DataLoader, Dataset, Subset
import torch

mfp_bc_fn = 'data/ASP_dataset_slices/all_cancer_256_mfp_bc0_comboscore.csv'
mfp_regcs_fn = 'data/ASP_dataset_slices/all_cancer_256_mfp_comboscore.csv'

In [2]:
# Test out balancing the classes
mfp_bc_bal_dataset = MorganFingerprintDataset(mfp_bc_fn, balance_classes=True)

# Check that the balance of positives and negatives is maintained
pos_count = 0
neg_count = 0
for i in range(len(mfp_bc_bal_dataset)):
    _, label = mfp_bc_bal_dataset[i]
    if label == 1:
        pos_count += 1
    else:
        neg_count += 1
assert pos_count == neg_count, f'Positives: {pos_count}, Negatives: {neg_count}'
print("Balanced classes maintained")
print("New class balanced dataset size:", len(mfp_bc_bal_dataset))

Original Dataset shape: (300928, 513)
Initial dataset size: 300928
Balancing classes
Balanced dataset size: 185604
Balanced classes maintained
New class balanced dataset size: 185604


In [3]:
# Test out nonbalanced classes
mfp_bc_unbal_dataset = MorganFingerprintDataset(mfp_bc_fn, balance_classes=False)
pos_count = 0
neg_count = 0
for i in range(len(mfp_bc_unbal_dataset)):
    _, label = mfp_bc_unbal_dataset[i]
    if label == 1:
        pos_count += 1
    else:
        neg_count += 1
assert pos_count != neg_count, f'Positives: {pos_count}, Negatives: {neg_count}'
print("No balancing of classes maintained")
print("Dataset size:", len(mfp_bc_unbal_dataset))

Original Dataset shape: (300928, 513)
No balancing of classes maintained
Dataset size: 300928


In [4]:
# Test out fails when balance classes is true but dataset is not binary
try:
    mfp_regcs_dataset = MorganFingerprintDataset(mfp_regcs_fn, balance_classes=True)
except AssertionError as e:
    print("Failed as expected")
    print(e)

Original Dataset shape: (300928, 513)
Initial dataset size: 300928
Balancing classes
Failed as expected
Labels should be binary if balancing classes


In [5]:
# Test works on regression when balance classes is false
mfp_regcs_dataset = MorganFingerprintDataset(mfp_regcs_fn, balance_classes=False)
print("Regression dataset size:", len(mfp_regcs_dataset))

Original Dataset shape: (300928, 513)
Regression dataset size: 300928


In [6]:
# Test splitting dataset code
bal_train, bal_val, bal_test = split_data(mfp_bc_bal_dataset, train_size=0.8, tune_size=0.1, test_size=0.1, random_state=42)

# Count the number of positives and negatives in each split
pos_train = 0
neg_train = 0
for i in range(len(bal_train)):
    _, label = bal_train[i]
    if label == 1:
        pos_train += 1
    else:
        neg_train += 1
print("Balanced train split")
print("Positives:", pos_train / (pos_train + neg_train), "Negatives:", neg_train / (pos_train + neg_train))

pos_val = 0
neg_val = 0
for i in range(len(bal_val)):
    _, label = bal_val[i]
    if label == 1:
        pos_val += 1
    else:
        neg_val += 1
print("Balanced val split")
print("Positives:", pos_val / (pos_val + neg_val), "Negatives:", neg_val / (pos_val + neg_val))

pos_test = 0
neg_test = 0
for i in range(len(bal_test)):
    _, label = bal_test[i]
    if label == 1:
        pos_test += 1
    else:
        neg_test += 1
print("Balanced test split")
print("Positives:", pos_test / (pos_test + neg_test), "Negatives:", neg_test / (pos_test + neg_test))


Train size: 148483
Tune size: 18560
Test size: 18561
Balanced train split
Positives: 0.5004141888296977 Negatives: 0.4995858111703023
Balanced val split
Positives: 0.4991379310344828 Negatives: 0.5008620689655172
Balanced test split
Positives: 0.49754862345778783 Negatives: 0.5024513765422122


In [7]:
# Test splitting dataset code, but in unbalanced dataset
unbal_train, unbal_val, unbal_test = split_data(mfp_bc_unbal_dataset, train_size=0.8, tune_size=0.1, test_size=0.1, random_state=42)

# Count the number of positives and negatives in each split
pos_train = 0
neg_train = 0
for i in range(len(unbal_train)):
    _, label = unbal_train[i]
    if label == 1:
        pos_train += 1
    else:
        neg_train += 1
print("Unbalanced train split")
print("Positives:", pos_train / (pos_train + neg_train), "Negatives:", neg_train / (pos_train + neg_train))

pos_val = 0
neg_val = 0
for i in range(len(unbal_val)):
    _, label = unbal_val[i]
    if label == 1:
        pos_val += 1
    else:
        neg_val += 1
print("Unbalanced val split")
print("Positives:", pos_val / (pos_val + neg_val), "Negatives:", neg_val / (pos_val + neg_val))

pos_test = 0
neg_test = 0
for i in range(len(unbal_test)):
    _, label = unbal_test[i]
    if label == 1:
        pos_test += 1
    else:
        neg_test += 1
print("Unbalanced test split")
print("Positives:", pos_test / (pos_test + neg_test), "Negatives:", neg_test / (pos_test + neg_test))


Train size: 240742
Tune size: 30093
Test size: 30093
Unbalanced train split
Positives: 0.30843392511485324 Negatives: 0.6915660748851468
Unbalanced val split
Positives: 0.30621739274914433 Negatives: 0.6937826072508557
Unbalanced test split
Positives: 0.3101718007510052 Negatives: 0.6898281992489947


Test getting the filename for the all cancer data - get_all_cancer_dataset_filename

In [8]:
test_mfp_bc0 = get_all_cancer_dataset_filename(use_mfp=True, use_dna=False, use_rna=False, use_prot=False, use_bc=True, use_csreg=False, use_pgreg=False, mfp_len=256, bc_cutoff=0)
test_mfpdna_bc0 = get_all_cancer_dataset_filename(use_mfp=True, use_dna=True, use_rna=False, use_prot=False, use_bc=True, use_csreg=False, use_pgreg=False, mfp_len=256, bc_cutoff=0)
test_mfprnaprot_csreg0 = get_all_cancer_dataset_filename(use_mfp=True, use_dna=False, use_rna=True, use_prot=True, use_bc=False, use_csreg=True, use_pgreg=False, mfp_len=256)
test_dnarnaprot_pgreg = get_all_cancer_dataset_filename(use_mfp=False, use_dna=True, use_rna=True, use_prot=True, use_bc=False, use_csreg=False, use_pgreg=True)

mfp_bc_fn = 'data/ASP_dataset_slices/all_cancer_256_mfp_bc0_comboscore.csv'
mfpdna_bc_fn = 'data/ASP_dataset_slices/all_cancer_256_mfpdna_bc0_comboscore.csv'
mfprnaprot_csreg0_fn = 'data/ASP_dataset_slices/all_cancer_256_mfprnaprot_comboscore.csv'
dnarnaprot_pgreg_fn = 'data/ASP_dataset_slices/all_cancer_dnarnaprot_percgrowth.csv'

assert test_mfp_bc0 == mfp_bc_fn, print('mfp_bc_fn error: ' + test_mfp_bc0 + ' vs ' + mfp_bc_fn)
assert test_mfpdna_bc0 == mfpdna_bc_fn, print('mfpdna_bc_fn error: ' + test_mfpdna_bc0 + ' vs ' + mfpdna_bc_fn)
assert test_mfprnaprot_csreg0 == mfprnaprot_csreg0_fn, print('mfprnaprot_csreg0_fn error: ' + test_mfprnaprot_csreg0 + ' vs ' + mfprnaprot_csreg0_fn)
assert test_dnarnaprot_pgreg == dnarnaprot_pgreg_fn, print('dnarnaprot_pgreg_fn error: ' + test_dnarnaprot_pgreg + ' vs ' + dnarnaprot_pgreg_fn)

Using filename:  data/ASP_dataset_slices/all_cancer_256_mfp_bc0_comboscore.csv
Using filename:  data/ASP_dataset_slices/all_cancer_256_mfpdna_bc0_comboscore.csv
Using filename:  data/ASP_dataset_slices/all_cancer_256_mfprnaprot_comboscore.csv
Using filename:  data/ASP_dataset_slices/all_cancer_dnarnaprot_percgrowth.csv


Test out filtering dataset based on cancer type

In [9]:
breast_comboscore_indices = 'data/ASP_dataset_slices/breast_comboscore_indices.txt'
test_breast_fn = get_cancer_type_indices_filename(cancer_type='breast', use_bc=False, use_csreg=True, use_pgreg=False)
assert breast_comboscore_indices == test_breast_fn, print('breast_comboscore_indices error: ' + breast_comboscore_indices + ' vs ' + test_breast_fn)
colon_pgreg_indices = 'data/ASP_dataset_slices/colon_pg_indices.txt'
test_colon_fn = get_cancer_type_indices_filename('colon', use_bc=False, use_csreg=False, use_pgreg=True)
assert colon_pgreg_indices == test_colon_fn, print('colon_pgreg_indices error: ' + colon_pgreg_indices + ' vs ' + test_colon_fn)

try:
    get_cancer_type_indices_filename('unknown', use_bc=False, use_csreg=False, use_pgreg=True)
except ValueError:
    print("Failed as expected 0")
try:
    get_cancer_type_indices_filename('breast', use_bc=False, use_csreg=True, use_pgreg=True)
except ValueError:
    print("Failed as expected 1")
try:
    get_cancer_type_indices_filename('breast', use_bc=True, use_csreg=False, use_pgreg=True)
except ValueError:
    print("Failed as expected 2")
try:
    get_cancer_type_indices_filename('breast', use_bc=False, use_csreg=False, use_pgreg=False)
except ValueError:
    print("Failed as expected 3")

mfp_breast_unbal_dataset = MorganFingerprintDataset(mfp_bc_fn, indices_filter_fn=breast_comboscore_indices)
print("Dataset size:", len(mfp_breast_unbal_dataset))

Failed as expected 0
Failed as expected 1
Failed as expected 2
Failed as expected 3
Original Dataset shape: (300928, 513)
Reading indices from data/ASP_dataset_slices/breast_comboscore_indices.txt
Filtered Dataset shape: (20989, 513)
Dataset size: 20989


Test out filtering dataset based on targeted-other drug pair classes

In [10]:
targeted_other_pg_indices = 'data/ASP_dataset_slices/all_cancer_targeted_other_pg_indices.txt'
test_target_other_pg_fn = get_drug_class_indices_filename('targeted_other', use_bc=False, use_csreg=False, use_pgreg=True)
assert targeted_other_pg_indices == test_target_other_pg_fn, print('targeted_other_pg_indices error: ' + targeted_other_pg_indices + ' vs ' + test_target_other_pg_fn)
targeted_targeted_cs_indices = 'data/ASP_dataset_slices/all_cancer_targeted_targeted_cs_indices.txt'
test_target_targeted_cs_fn = get_drug_class_indices_filename('targeted_targeted', use_bc=False, use_csreg=True, use_pgreg=False)
assert targeted_targeted_cs_indices == test_target_targeted_cs_fn, print('targeted_targeted_cs_indices error: ' + targeted_targeted_cs_indices + ' vs ' + test_target_targeted_cs_fn)

try:
    get_drug_class_indices_filename('unknown', use_bc=False, use_csreg=False, use_pgreg=True)
except ValueError:
    print("Failed as expected 0")
try:
    get_drug_class_indices_filename('chemo_chemo', use_bc=False, use_csreg=True, use_pgreg=True)
except ValueError:
    print("Failed as expected 1")
try:
    get_drug_class_indices_filename('chemo_other', use_bc=True, use_csreg=False, use_pgreg=True)
except ValueError:
    print("Failed as expected 2")
try:
    get_drug_class_indices_filename('other_other', use_bc=False, use_csreg=False, use_pgreg=False)
except ValueError:
    print("Failed as expected 3")

mfprna_pg_fn = 'data/ASP_dataset_slices/all_cancer_256_mfprna_percgrowth.csv'
mfprna_targeted_other_pg_unbal_dataset = MorganFingerprintDataset(mfprna_pg_fn, indices_filter_fn=targeted_other_pg_indices)
print("Dataset size:", len(mfprna_targeted_other_pg_unbal_dataset))

Failed as expected 0
Failed as expected 1
Failed as expected 2
Failed as expected 3
Original Dataset shape: (2774280, 1237)
Reading indices from data/ASP_dataset_slices/all_cancer_targeted_other_pg_indices.txt
Filtered Dataset shape: (214848, 1237)
Dataset size: 214848


Test get_mask1gl

In [11]:
output_prefix='scratch/anticancer-synergy-prediction-scratch/experiments/20240820_dataset_test/'
mask_test_cs = get_mask1gl(True, True, True, True, output_prefix+'mask_cs.csv', True, False)
assert mask_test_cs.shape == torch.Size([722, 3127]), print('mask_test.shape error: ' + str(mask_test_cs.shape))

mask_test_pgreg = get_mask1gl(True, True, True, True, output_prefix+'mask_pgreg.csv', False, True)
assert mask_test_pgreg.shape == torch.Size([722, 3129]), print('mask_test.shape error: ' + str(mask_test_pgreg.shape))

# Check that mask does not use both cs and pgreg
try: 
    mask_test_mfptrue_both = get_mask1gl(True, True, True, True, output_prefix+'mask_mfptrue_both.csv', True, True)
except ValueError as e:
    print("Failed mfp true, cs and pg both true, as expected")
    print(e)

# Check that mask cannot use neither cs and pgreg if mfp is true
try: 
    mask_test_mfptrue_neither = get_mask1gl(True, True, True, True, output_prefix+'mask_mfptrue_neither.csv', False, False)
except ValueError as e:
    print("Failed mfp true, cs and pg both false, as expected")
    print(e)

# Check that mask is okay to use neithr or both cs and pgreg if mfp is false
mask_test_mfpfalse_neither = get_mask1gl(False, True, True, True, output_prefix+'mask_mfpfalse_neither.csv', False, False)
assert mask_test_mfpfalse_neither.shape == torch.Size([722, 2615]), print('mask_test.shape error: ' + str(mask_test_mfpfalse_neither.shape))
mask_test_mfpfalse_both = get_mask1gl(False, True, True, True, output_prefix+'mask_mfpfalse_both.csv', True, True)
assert mask_test_mfpfalse_both.shape == torch.Size([722, 2615]), print('mask_test.shape error: ' + str(mask_test_mfpfalse_both.shape))



torch.Size([722, 3127])
torch.Size([722, 3129])
Failed mfp true, cs and pg both true, as expected
Cannot use both cs and pg or neither, choose whether to include conc
Failed mfp true, cs and pg both false, as expected
Cannot use both cs and pg or neither, choose whether to include conc
torch.Size([722, 2615])
torch.Size([722, 2615])


Test get_masks_concat

In [12]:
# Test the get_masks_concat function
# Error checks
try:
    get_masks_concat(use_mfp=False, use_dna=False, use_rna=False, use_prot=False, use_bc=False, use_csreg=False, use_pgreg=False)
except ValueError as e:
    print("Failed as expected: ", e)

try:
    get_masks_concat(use_mfp=False, use_dna=False, use_rna=False, use_prot=False, use_bc=True, use_csreg=False, use_pgreg=False)
except ValueError as e:
    print("Failed as expected: ", e)

try:
    get_masks_concat(use_mfp=True, use_dna=False, use_rna=False, use_prot=False, use_bc=True, use_csreg=True, use_pgreg=False)
except ValueError as e:
    print("Failed as expected: ", e)

# Single Omics
masks_mfp_bc0 = get_masks_concat(use_mfp=True, use_dna=False, use_rna=False, use_prot=False, use_bc=True, use_csreg=False, use_pgreg=False)
assert len(masks_mfp_bc0) == 1, print('masks_mfp_bc0 length error: ' + str(len(masks_mfp_bc0)))
assert masks_mfp_bc0[0].shape == torch.Size([512, 1]), print('masks_mfp_bc0 shape error: ' + str(masks_mfp_bc0[0].shape))

masks_mfp_cs = get_masks_concat(use_mfp=True, use_dna=False, use_rna=False, use_prot=False, use_bc=False, use_csreg=True, use_pgreg=False)
assert len(masks_mfp_cs) == 1, print('masks_mfp_cs length error: ' + str(len(masks_mfp_cs)))
assert masks_mfp_cs[0].shape == torch.Size([512, 1]), print('masks_mfp_cs shape error: ' + str(masks_mfp_cs[0].shape))

masks_mfp_pg = get_masks_concat(use_mfp=True, use_dna=False, use_rna=False, use_prot=False, use_bc=False, use_csreg=False, use_pgreg=True)
assert len(masks_mfp_pg) == 1, print('masks_mfp_pg length error: ' + str(len(masks_mfp_pg)))
assert masks_mfp_pg[0].shape == torch.Size([514, 1]), print('masks_mfp_pg shape error: ' + str(masks_mfp_pg[0].shape))

masks_dna_bc0 = get_masks_concat(use_mfp=False, use_dna=True, use_rna=False, use_prot=False, use_bc=True, use_csreg=False, use_pgreg=False)
assert len(masks_dna_bc0) == 1, print('masks_dna_bc0 length error: ' + str(len(masks_dna_bc0)))
assert masks_dna_bc0[0].shape == torch.Size([1171, 1]), print('masks_dna_bc0 shape error: ' + str(masks_dna_bc0[0].shape))

masks_rna_bc0 = get_masks_concat(use_mfp=False, use_dna=False, use_rna=True, use_prot=False, use_bc=True, use_csreg=False, use_pgreg=False)
assert len(masks_rna_bc0) == 1, print('masks_rna_bc0 length error: ' + str(len(masks_rna_bc0)))
assert masks_rna_bc0[0].shape == torch.Size([722, 1]), print('masks_rna_bc0 shape error: ' + str(masks_rna_bc0[0].shape))

masks_prot_bc0 = get_masks_concat(use_mfp=False, use_dna=False, use_rna=False, use_prot=True, use_bc=True, use_csreg=False, use_pgreg=False)
assert len(masks_prot_bc0) == 1, print('masks_prot_bc0 length error: ' + str(len(masks_prot_bc0)))
assert masks_prot_bc0[0].shape == torch.Size([722, 1]), print('masks_prot_bc0 shape error: ' + str(masks_prot_bc0[0].shape))

# Two Omics
masks_mfpdna_bc0 = get_masks_concat(use_mfp=True, use_dna=True, use_rna=False, use_prot=False, use_bc=True, use_csreg=False, use_pgreg=False)
assert len(masks_mfpdna_bc0) == 2, print('masks_mfpdna_bc0 length error: ' + str(len(masks_mfpdna_bc0)))
assert masks_mfpdna_bc0[0].shape == torch.Size([512, 1171]), print('masks_mfpdna_bc0 shape error: ' + str(masks_mfpdna_bc0[0].shape))
assert masks_mfpdna_bc0[1].shape == torch.Size([1171, 1]), print('masks_mfpdna_bc0 shape error: ' + str(masks_mfpdna_bc0[1].shape))

masks_mfpdna_pg = get_masks_concat(use_mfp=True, use_dna=True, use_rna=False, use_prot=False, use_bc=False, use_csreg=False, use_pgreg=True)
assert len(masks_mfpdna_pg) == 2, print('masks_mfpdna_pg length error: ' + str(len(masks_mfpdna_pg)))
assert masks_mfpdna_pg[0].shape == torch.Size([514, 1171]), print('masks_mfpdna_pg shape error: ' + str(masks_mfpdna_pg[0].shape))
assert masks_mfpdna_pg[1].shape == torch.Size([1171, 1]), print('masks_mfpdna_pg shape error: ' + str(masks_mfpdna_pg[1].shape))

masks_mfprna_bc0 = get_masks_concat(use_mfp=True, use_dna=False, use_rna=True, use_prot=False, use_bc=True, use_csreg=False, use_pgreg=False)
assert len(masks_mfprna_bc0) == 2, print('masks_mfprna_bc0 length error: ' + str(len(masks_mfprna_bc0)))
assert masks_mfprna_bc0[0].shape == torch.Size([512, 722]), print('masks_mfprna_bc0 shape error: ' + str(masks_mfprna_bc0[0].shape))
assert masks_mfprna_bc0[1].shape == torch.Size([722, 1]), print('masks_mfprna_bc0 shape error: ' + str(masks_mfprna_bc0[1].shape))

masks_mfpprot_bc0 = get_masks_concat(use_mfp=True, use_dna=False, use_rna=False, use_prot=True, use_bc=True, use_csreg=False, use_pgreg=False)
assert len(masks_mfpprot_bc0) == 2, print('masks_mfpprot_bc0 length error: ' + str(len(masks_mfpprot_bc0)))
assert masks_mfpprot_bc0[0].shape == torch.Size([512, 722]), print('masks_mfpprot_bc0 shape error: ' + str(masks_mfpprot_bc0[0].shape))
assert masks_mfpprot_bc0[1].shape == torch.Size([722, 1]), print('masks_mfpprot_bc0 shape error: ' + str(masks_mfpprot_bc0[1].shape))

# Three Omics
masks_mfpdnarna_bc0 = get_masks_concat(use_mfp=True, use_dna=True, use_rna=True, use_prot=False, use_bc=True, use_csreg=False, use_pgreg=False)
assert len(masks_mfpdnarna_bc0) == 3, print('masks_mfpdnarna_bc0 length error: ' + str(len(masks_mfpdnarna_bc0)))
assert masks_mfpdnarna_bc0[0].shape == torch.Size([512, 1171]), print('masks_mfpdnarna_bc0 shape error: ' + str(masks_mfpdnarna_bc0[0].shape))
assert masks_mfpdnarna_bc0[1].shape == torch.Size([1171, 722]), print('masks_mfpdnarna_bc0 shape error: ' + str(masks_mfpdnarna_bc0[1].shape))
assert masks_mfpdnarna_bc0[2].shape == torch.Size([722, 1]), print('masks_mfpdnarna_bc0 shape error: ' + str(masks_mfpdnarna_bc0[2].shape))
# Check if the number of 1s in the dna to rna mask is 50684
assert masks_mfpdnarna_bc0[1].sum() == 50684, print('masks_mfpdnarna_bc0 sum error: ' + str(masks_mfpdnarna_bc0[1].sum()))

# All omics
masks_mfpdnarnaprot_cs = get_masks_concat(use_mfp=True, use_dna=True, use_rna=True, use_prot=True, use_bc=False, use_csreg=True, use_pgreg=False)
assert len(masks_mfpdnarnaprot_cs) == 4, print('masks_mfpdnarnaprot_cs length error: ' + str(len(masks_mfpdnarnaprot_cs)))
assert masks_mfpdnarnaprot_cs[0].shape == torch.Size([512, 1171]), print('masks_mfpdnarnaprot_cs shape error: ' + str(masks_mfpdnarnaprot_cs[0].shape))
assert masks_mfpdnarnaprot_cs[1].shape == torch.Size([1171, 722]), print('masks_mfpdnarnaprot_cs shape error: ' + str(masks_mfpdnarnaprot_cs[1].shape))
assert masks_mfpdnarnaprot_cs[2].shape == torch.Size([722, 722]), print('masks_mfpdnarnaprot_cs shape error: ' + str(masks_mfpdnarnaprot_cs[2].shape))
assert masks_mfpdnarnaprot_cs[3].shape == torch.Size([722, 1]), print('masks_mfpdnarnaprot_cs shape error: ' + str(masks_mfpdnarnaprot_cs[3].shape))

masks_mfpdnarnaprot_pg = get_masks_concat(use_mfp=True, use_dna=True, use_rna=True, use_prot=True, use_bc=False, use_csreg=False, use_pgreg=True)
assert len(masks_mfpdnarnaprot_pg) == 4, print('masks_mfpdnarnaprot_pg length error: ' + str(len(masks_mfpdnarnaprot_pg)))
assert masks_mfpdnarnaprot_pg[0].shape == torch.Size([514, 1171]), print('masks_mfpdnarnaprot_pg shape error: ' + str(masks_mfpdnarnaprot_pg[0].shape))
assert masks_mfpdnarnaprot_pg[1].shape == torch.Size([1171, 722]), print('masks_mfpdnarnaprot_pg shape error: ' + str(masks_mfpdnarnaprot_pg[1].shape))
assert masks_mfpdnarnaprot_pg[2].shape == torch.Size([722, 722]), print('masks_mfpdnarnaprot_pg shape error: ' + str(masks_mfpdnarnaprot_pg[2].shape))
assert masks_mfpdnarnaprot_pg[3].shape == torch.Size([722, 1]), print('masks_mfpdnarnaprot_pg shape error: ' + str(masks_mfpdnarnaprot_pg[3].shape))



Failed as expected:  Must use at least one prediction task
Failed as expected:  Must use at least one omics layer
Failed as expected:  Cannot use both two prediction tasks simultaneously
